for a simplistic training purpose, we'll use a simple neural network architecture

prep data

In [2]:
import torch

X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])

y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6]
])

y_test = torch.tensor([0, 1])

In [3]:
# defining custom dataset classes
from torch.utils.data import Dataset


class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y

    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y

    def __len__(self):
        return self.labels.shape[0]

In [6]:
train_ds=ToyDataset(X_train, y_train)
test_ds=ToyDataset(X_test, y_test)



In [7]:
from torch.utils.data import DataLoader
torch.manual_seed(123)

train_loader=DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

test_loader=DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)


In [19]:
for idx, (x,y) in enumerate(train_loader):
    print(f"Batch: {idx}:",x,y)

Batch: 0: tensor([[-0.5000,  2.6000],
        [-0.9000,  2.9000]]) tensor([0, 0])
Batch: 1: tensor([[ 2.3000, -1.1000],
        [-1.2000,  3.1000]]) tensor([1, 0])


In [8]:
# a simple neural network architecture

class NeuralNetwork(torch.nn.Module):
    def __init__(self,num_inputs, num_outputs):
        super().__init__()

        self.layers=torch.nn.Sequential(
            torch.nn.Linear(num_inputs,30),
            torch.nn.ReLU(),

            torch.nn.Linear(30,20),
            torch.nn.ReLU(),

            torch.nn.Linear(20,num_outputs)
        )

    def forward(self,x):
        logits=self.layers(x)
        return logits
    
    

In [18]:
# torch.manual_seed(123)
# model=NeuralNetwork(2,2)

# print(model) # model architecture

# # total number of parameters
# num_params=sum(p.numel() for p in model.parameters() if p.requires_grad)
# print(num_params)


*a typical training loop* 

In [ ]:
import torch.nn.functional as F
torch.manual_seed(123)

model=NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer=torch.optim.SGD(params=model.parameters(), lr=0.5)

epochs=3
for epoch in range(epochs):

    model.train() # model training mode

    for batch_idx, (features, labels) in enumerate(train_loader):
        # features is input
        # labels is expected outputs
        
        # forward pass
        logits=model(features)  

        # loss function
        loss=F.cross_entropy(logits, labels)

        # set the gradients from the previous round to zero to prevent
        # unintended gradient accumulation
        optimizer.zero_grad()

        # back propagatio 
        loss.backward()

        # optimizer use the gradients to update the model parameters
        optimizer.step()

        print(f"Epoch: {epoch}| Train Loss: {loss:.2f}")

    
    # for testing purpose
    model.eval() # model testing mode
    # we train on train_loader, test on test_loader



Epoch: 0| Train Loss: 0.75
Epoch: 0| Train Loss: 0.65
Epoch: 1| Train Loss: 0.44
Epoch: 1| Train Loss: 0.13
Epoch: 2| Train Loss: 0.03
Epoch: 2| Train Loss: 0.00


In [28]:
model.eval()
with torch.no_grad():
    outputs=model(X_train)
print(outputs)


tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


In [ ]:
# for class membership probablities
torch.set_printoptions(sci_mode=False)
probabs=torch.softmax(outputs, dim=1)
print(probabs)


tensor([[0.9991, 0.0009],
        [0.9982, 0.0018],
        [0.9949, 0.0051],
        [0.0491, 0.9509],
        [0.0307, 0.9693]])


In [30]:
# we can convert those class member ship probablities into labels
predictions=torch.argmax(probabs, dim=1)
print(predictions)


tensor([0, 0, 0, 1, 1])


In [31]:
# instead of passing through softmax, we can direct pass through argmax
predictions=torch.argmax(outputs, dim=1)
print(predictions)


tensor([0, 0, 0, 1, 1])


In [32]:
# double checking predictions with actual values
predictions==y_train

tensor([True, True, True, True, True])

In [34]:
# torch.sum count number of correct predictions
torch.sum(predictions==y_train)

tensor(5)

*a function to compute prediction accuracy*

In [35]:
def compute_accuracy(model, dataloader):
    model=model.eval()
    correct=0.0
    total_examples=0

    # feature : input
    # labels  : expected output

    for idx, (features, labels) in enumerate(dataloader):
        with torch.no_grad():
            logits=model(features)

        predictions=torch.argmax(logits,dim=1)
        compare=labels==predictions
        correct+=torch.sum(compare)
        total_examples+=len(compare)

    return (correct/total_examples).item()



In [36]:
print(compute_accuracy(model=model,dataloader=train_loader))


1.0


working with single GPU (if available)

In [38]:
device="cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [39]:
import torch.nn.functional as F
torch.manual_seed(123)

model = NeuralNetwork(num_inputs=2, num_outputs=2)
model=model.to(device=device) # setting up model for device

optimizer = torch.optim.SGD(params=model.parameters(), lr=0.5)

epochs = 3
for epoch in range(epochs):

    model.train()  # model training mode

    for batch_idx, (features, labels) in enumerate(train_loader):
        # features is input
        # labels is expected outputs

        # loading features, labels into device
        features, labels=features.to(device), labels.to(device)

        # forward pass
        logits = model(features)

        # loss function
        loss = F.cross_entropy(logits, labels)

        # set the gradients from the previous round to zero to prevent
        # unintended gradient accumulation
        optimizer.zero_grad()

        # back propagatio
        loss.backward()

        # optimizer use the gradients to update the model parameters
        optimizer.step()

        print(f"Epoch: {epoch}| Train Loss: {loss:.2f}")

    # for testing purpose
    model.eval()  # model testing mode
    # we train on train_loader, test on test_loader

    

Epoch: 0| Train Loss: 0.75
Epoch: 0| Train Loss: 0.65
Epoch: 1| Train Loss: 0.44
Epoch: 1| Train Loss: 0.13
Epoch: 2| Train Loss: 0.03
Epoch: 2| Train Loss: 0.00
